In [1]:
#API connection and dependency install
import os
from flipside import Flipside
from dotenv import find_dotenv, load_dotenv
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Confirm and define dotenv path
dotenv_path = find_dotenv()
print(f"Loading .env from: {dotenv_path}")

#load env data from path
load_dotenv(dotenv_path, override=True)

#store API_KEY from dotenv file
import os
API_KEY = os.getenv("FLIP_API_KEY")
# print(f"API Key loaded: {API_KEY}")

#instantiate flipside client
flipside = Flipside(API_KEY, "https://api-v2.flipsidecrypto.xyz")

Loading .env from: c:\Users\savan\Desktop\portfolio_repo\arb_airdrop\.env


In [12]:
import os

os.makedirs('data/processed/transfers_batches', exist_ok=True)


Load airdrop recipient wallets as a list. Query wallet transfers in batches from ez_token_transfers

In [13]:
recipients_df = pd.read_csv(r"C:\Users\savan\Desktop\portfolio_repo\arb_airdrop\data\processed\arb_drop_data")

batch_size = 1000
wallet_list = recipients_df['user_address'].unique()

for i in range(0, len(wallet_list), batch_size):
    batch = wallet_list[i:i+batch_size].tolist()
    batch_str = "','".join(batch)
    
    sql = f"""
        SELECT block_number, block_timestamp, contract_address, from_address, to_address, amount
        FROM arbitrum.core.ez_token_transfers
        WHERE (from_address IN ('{batch_str}') OR to_address IN ('{batch_str}'))
        AND contract_address = LOWER('0x912CE59144191C1204E64559FE8253a0e49E6548')
    """
    
    wallet_transfers = flipside.query(sql)
    
    columns = wallet_transfers.columns
    rows = wallet_transfers.rows  
    wallet_transfers_df = pd.DataFrame(rows, columns=columns)
    wallet_transfers_df.to_parquet(f"data/processed/transfers_batches/transfers_batch_{i}.parquet", index=False)
    print(f"Saved batch {i} to {i+batch_size} with {len(wallet_transfers_df)} rows")

Saved batch 0 to 1000 with 12159 rows
Saved batch 1000 to 2000 with 100000 rows
Saved batch 2000 to 3000 with 10183 rows
Saved batch 3000 to 4000 with 12103 rows
Saved batch 4000 to 5000 with 22875 rows
Saved batch 5000 to 6000 with 10072 rows
Saved batch 6000 to 7000 with 15367 rows
Saved batch 7000 to 8000 with 9428 rows
Saved batch 8000 to 9000 with 8871 rows
Saved batch 9000 to 10000 with 9360 rows
Saved batch 10000 to 11000 with 8172 rows
Saved batch 11000 to 12000 with 7735 rows
Saved batch 12000 to 13000 with 18951 rows
Saved batch 13000 to 14000 with 8089 rows
Saved batch 14000 to 15000 with 14336 rows
Saved batch 15000 to 16000 with 8362 rows
Saved batch 16000 to 17000 with 7140 rows
Saved batch 17000 to 18000 with 8973 rows
Saved batch 18000 to 19000 with 13986 rows
Saved batch 19000 to 20000 with 11196 rows
Saved batch 20000 to 21000 with 11072 rows
Saved batch 21000 to 22000 with 11750 rows
Saved batch 22000 to 23000 with 10164 rows
Saved batch 23000 to 24000 with 13286 row

Stitch together files

In [14]:
import glob

In [ ]:
import os
import glob

base_dir = os.path.join(os.getcwd(), 'data', 'processed', 'transfers_batches')
pattern = os.path.join(base_dir, 'transfers_batch_*.parquet')

batch_files = glob.glob(pattern)
# print(f"Found files: {batch_files}")


Found files: ['c:\\Users\\savan\\Desktop\\portfolio_repo\\arb_airdrop\\notebooks\\data\\processed\\transfers_batches\\transfers_batch_0.parquet', 'c:\\Users\\savan\\Desktop\\portfolio_repo\\arb_airdrop\\notebooks\\data\\processed\\transfers_batches\\transfers_batch_1000.parquet', 'c:\\Users\\savan\\Desktop\\portfolio_repo\\arb_airdrop\\notebooks\\data\\processed\\transfers_batches\\transfers_batch_10000.parquet', 'c:\\Users\\savan\\Desktop\\portfolio_repo\\arb_airdrop\\notebooks\\data\\processed\\transfers_batches\\transfers_batch_11000.parquet', 'c:\\Users\\savan\\Desktop\\portfolio_repo\\arb_airdrop\\notebooks\\data\\processed\\transfers_batches\\transfers_batch_12000.parquet', 'c:\\Users\\savan\\Desktop\\portfolio_repo\\arb_airdrop\\notebooks\\data\\processed\\transfers_batches\\transfers_batch_13000.parquet', 'c:\\Users\\savan\\Desktop\\portfolio_repo\\arb_airdrop\\notebooks\\data\\processed\\transfers_batches\\transfers_batch_14000.parquet', 'c:\\Users\\savan\\Desktop\\portfolio_r

In [21]:
# batch_files = glob.glob("data/processed/transfers_batches/transfers_batch_*.parquet")
transfers_dfs = [pd.read_parquet(f) for f in batch_files]
transfers_df = pd.concat(transfers_dfs, ignore_index=True)
print(f"Combined all transfer batches: {len(transfers_df)} rows")

Combined all transfer batches: 1213667 rows


In [ ]:
#clean up needed col headers for merge
recipients_df = recipients_df.rename(columns={
    'user_address': 'wallet_address',
    'block_timestamp': 'claim_time',
    'amount': 'claimed_amount'
})
print(recipients_df.columns)

Index(['blockchain', 'airdrop', 'action_type', 'tx_hash', 'claim_time',
       'contract_address', 'wallet_address', 'claimed_amount', 'amount_usd',
       '__row_index', 'block_date'],
      dtype='object')


Merge and Filter to 30 Days Post Claim

In [ ]:
# merged_df = transfers_df.merge(recipients_df[['wallet_address', 'claim_time']], 
#                                left_on='from_address', right_on='wallet_address', how='inner')

# # Filter within 30 days of claim
# merged_df = merged_df[
#     (merged_df['block_timestamp'] >= merged_df['claim_time']) &
#     (merged_df['block_timestamp'] < merged_df['claim_time'] + pd.Timedelta(days=30))
# ]

# # Calculate balance delta
# merged_df['delta'] = merged_df.apply(
#     lambda row: -row['amount'] if row['from_address'] == row['wallet_address'] else row['amount'], axis=1
# )

# print(f"✅ Filtered to {len(merged_df)} transfers within 30-day windows")

In [29]:
# Merge on from_address → capturing send events
from_merge = transfers_df.merge(
    recipients_df[['wallet_address', 'claim_time']],
    left_on='from_address',
    right_on='wallet_address',
    how='inner'
)
from_merge['direction'] = 'sent'  # Mark direction

# Merge on to_address → capturing receive events
to_merge = transfers_df.merge(
    recipients_df[['wallet_address', 'claim_time']],
    left_on='to_address',
    right_on='wallet_address',
    how='inner'
)
to_merge['direction'] = 'received'  # Mark direction

merged_df = pd.concat([from_merge, to_merge], ignore_index=True)


Delta Calculation

In [30]:
merged_df['delta'] = merged_df.apply(
    lambda row: -row['amount'] if row['direction'] == 'sent' else row['amount'], axis=1
)

Daily Balances

In [32]:
daily_balances = merged_df.groupby(['wallet_address', 'block_timestamp']).agg({'delta': 'sum'}).reset_index()
daily_balances['cumulative_balance'] = daily_balances.groupby('wallet_address')['delta'].cumsum()

In [55]:
# daily_balances.head(200)

daily_balances.to_csv(r"C:\Users\savan\Desktop\portfolio_repo\arb_airdrop\data\processed\daily_balances.csv")

Balances after 30 days

In [34]:
recipients_df['claim_time'] = pd.to_datetime(recipients_df['claim_time'])
daily_balances['block_timestamp'] = pd.to_datetime(daily_balances['block_timestamp'])

In [35]:
# Merge to get claim_time and claimed_amount in daily_balances
merged_df = daily_balances.merge(
    recipients_df[['wallet_address', 'claim_time', 'claimed_amount']],
    on='wallet_address',
    how='left'
)

In [38]:
# Ensure all timestamps are UTC-aware and comparable
merged_df['block_timestamp'] = pd.to_datetime(merged_df['block_timestamp'], utc=True)
merged_df['claim_time'] = pd.to_datetime(merged_df['claim_time'], utc=True)
merged_df['day_30'] = merged_df['claim_time'] + pd.Timedelta(days=30)

In [50]:
# Filter rows within the 30-day window
filtered_df = merged_df[merged_df['block_timestamp'] <= merged_df['day_30']]

# Keep the latest record per wallet before day 30
latest_balance = filtered_df.sort_values(['wallet_address', 'block_timestamp']) \
    .groupby('wallet_address').tail(1)

# Select relevant columns
final_df = latest_balance[['wallet_address', 'claimed_amount', 'cumulative_balance', 'block_timestamp']]

print(final_df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 100000 entries, 37 to 1154241
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   wallet_address      100000 non-null  object             
 1   claimed_amount      100000 non-null  int64              
 2   cumulative_balance  100000 non-null  float64            
 3   block_timestamp     100000 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), float64(1), int64(1), object(1)
memory usage: 3.8+ MB
None


In [51]:
display(latest_balance.head())

,wallet_address,block_timestamp,delta,cumulative_balance,claim_time,claimed_amount,day_30
37,0x00000000009a41862f3b2b0c688b7c0d1940511e,2023-03-23 14:51:03+00:00,0.0,0.0,2023-03-23 13:38:43+00:00,4250,2023-04-22 13:38:43+00:00
41,0x0000000000dfd67ffd6c24251348f7c4f933cab4,2023-03-23 13:28:56+00:00,-1750.0,0.0,2023-03-23 13:28:55+00:00,1750,2023-04-22 13:28:55+00:00
43,0x0000000000e189dd664b9ab08a33c4839953852c,2023-03-23 13:26:55+00:00,-2250.0,0.0,2023-03-23 13:26:41+00:00,2250,2023-04-22 13:26:41+00:00
45,0x000000000279ef217428b1c3906ec8124784b70f,2023-03-23 13:28:01+00:00,-3250.0,0.0,2023-03-23 13:27:58+00:00,3250,2023-04-22 13:27:58+00:00
48,0x0000000009572a244a6c2d06ffe7be30e3bd2aec,2023-04-05 07:13:32+00:00,10.0,10.0,2023-03-23 18:41:59+00:00,625,2023-04-22 18:41:59+00:00


In [43]:
display(final_df.head())

,wallet_address,claimed_amount,cumulative_balance,block_timestamp
37,0x00000000009a41862f3b2b0c688b7c0d1940511e,4250,0.0,2023-03-23 14:51:03+00:00
41,0x0000000000dfd67ffd6c24251348f7c4f933cab4,1750,0.0,2023-03-23 13:28:56+00:00
43,0x0000000000e189dd664b9ab08a33c4839953852c,2250,0.0,2023-03-23 13:26:55+00:00
45,0x000000000279ef217428b1c3906ec8124784b70f,3250,0.0,2023-03-23 13:28:01+00:00
48,0x0000000009572a244a6c2d06ffe7be30e3bd2aec,625,10.0,2023-04-05 07:13:32+00:00


In [47]:
display(merged_df.head())

,wallet_address,block_timestamp,delta,cumulative_balance,claim_time,claimed_amount,day_30
0,0x00000000009a41862f3b2b0c688b7c0d1940511e,2023-03-21 23:34:22+00:00,0.0,0.0,2023-03-23 13:38:43+00:00,4250,2023-04-22 13:38:43+00:00
1,0x00000000009a41862f3b2b0c688b7c0d1940511e,2023-03-21 23:34:23+00:00,0.0,0.0,2023-03-23 13:38:43+00:00,4250,2023-04-22 13:38:43+00:00
2,0x00000000009a41862f3b2b0c688b7c0d1940511e,2023-03-21 23:34:24+00:00,0.0,0.0,2023-03-23 13:38:43+00:00,4250,2023-04-22 13:38:43+00:00
3,0x00000000009a41862f3b2b0c688b7c0d1940511e,2023-03-22 21:24:45+00:00,0.0,0.0,2023-03-23 13:38:43+00:00,4250,2023-04-22 13:38:43+00:00
4,0x00000000009a41862f3b2b0c688b7c0d1940511e,2023-03-22 21:24:46+00:00,0.0,0.0,2023-03-23 13:38:43+00:00,4250,2023-04-22 13:38:43+00:00
